In [ ]:
import transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn

# model_name = 'FacebookAI/xlm-roberta-base'
model_name = 'distilbert/distilbert-base-multilingual-cased'
train_data = pd.read_csv("train.csv", encoding='latin-1')
val_data = pd.read_csv("val.csv", encoding='latin-1')
test_data = pd.read_csv("test.csv", encoding='latin-1')

def tupleize(data, max_length=512):
    data['safe'] = data['safe'].astype(int)
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    
    tokens = tokenizer(list(data["prompt"]), return_tensors="np", padding=True, truncation=True, max_length=max_length)
    
    token_lengths = [len(token) for token in tokens['input_ids']]
    data_filtered = data.iloc[np.where(np.array(token_lengths) <= max_length)]
    
    tokens = tokenizer(list(data_filtered["prompt"]), return_tensors="np", padding=True, truncation=True, max_length=max_length)
    
    labels = np.array(data_filtered["safe"])
    
    return (tokens, labels)

train_data = tupleize(train_data)
val_data = tupleize(val_data)
test_data = tupleize(test_data)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(train_data[1])
plt.hist(val_data[1])
plt.hist(test_data[1])

In [ ]:
import transformers
import tensorflow as tf
import datasets

model = transformers.TFAutoModelForSequenceClassification.from_pretrained(model_name)
model.compile(optimizer=transformers.AdamWeightDecay(learning_rate=3e-5), metrics=['accuracy'])  # No loss argument!
model.fit(x=train_data[0], y=train_data[1], validation_data=(val_data[0], val_data[1]), batch_size=8, epochs=5)

In [ ]:
model.evaluate(test_data[0], test_data[1])

In [ ]:
model.save('model_ttv.keras')

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model.predict(tokenizer("Pano magluto ng sinigang?", return_tensors="np"))